### 좋아요수 / 댓글 분산 적용 기반 추천(명곡?)
- Input(title, artist) : contains로 찾기 => 결과 songId 한 개만 반환하도록 한다.
- I am 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 -> 가사분석해서 비슷한 노래 찾기 top5
- I am 찾기 -> 가사분석해서 비슷한 노래 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 top5
- 반환된 songId 들의 정보 출력

In [10]:
import numpy as np
import pandas as pd
import joblib, re, string

In [11]:
df = pd.read_csv('data/melon_song.csv')
# cosine_sim = joblib.load('data/melon_cosine_sim.sim')
df.tail(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
6245,34183979,인생 한잔,정은지,포크/블루스 국내드라마,술꾼도시여자들 OST Part 2,빨간양말,INAN,20211029.0,https://cdnimg.melon.co.kr/cm2/album/images/10...,12,4512,사랑도 이별도 오늘은 잠시 미뤄보자\n불안한 청춘도 내일로 잠시 미뤄두자\n떠나버린...,506225967 434154554
6246,31191637,6 o'clock,볼빨간사춘기,인디음악 포크/블루스,Red Diary 'Hidden Track',안지영,안지영,20180717.0,https://cdnimg.melon.co.kr/cm/album/images/101...,87,15668,It's 6 o'clock\n너와 나는 시간이 빨라서\n시간이 달라서\n어쩔 수 없...,458080033 442957041


In [3]:
df.isna().sum()

songId         0
title          0
artist         0
genre          2
album          0
lyricist     398
composer     402
date          45
img            0
comment        0
like           0
lyric          0
plylstSeq    121
dtype: int64

In [8]:
df.head(1)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,418168,희재,성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201.0,https://cdnimg.melon.co.kr/cm/album/images/000...,332,138267,햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고...,445029956 411111859


In [12]:
# 데이터 처리를 위한 작업
df.date.fillna(0, inplace=True)
df['date'] = df.date.astype(int).astype(str)
df.fillna('', inplace=True)
df['comment_like_total'] = df.comment + df.like
df['songId'] = df.songId.astype(str)

In [29]:
# 한글 불용어 처리
with open('data/한글불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]
stop_words.extend('은 는 를 도 을 며 의 에 게 니 거 로 요 과 래 랑 파 여 에게'.split())

- 이상치 songId : 1772803,
- 테스트해보고 싶은 것들 : 35456588


In [31]:
from konlpy.tag import Okt
okt = Okt()

In [32]:
def morphs_process(text):
    lyrics = []
    for lyric in text:
        lyric = lyric.replace('\n', ' ')
        morphs = okt.morphs(lyric, stem=True)
        tmp = [word for word in morphs if word not in stop_words]
        lyrics.append(' '.join(tmp))
    return lyrics

In [33]:
test_lyric = df[df.songId == '35456588'].lyric.values[0]
# print(test_lyric)
test_lyric = re.sub('[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣\n ]', '', test_lyric)
test_lyric = test_lyric.replace('\n \n', '\n')
morphs_process([test_lyric])

['친구 모두 모이다 여기 여기 붙다 jump 신나다 발 굳다 jump jump jump jump jump 엄마 아빠 이모 고모 삼촌 뛰다 jump 누구 든 상관없다 jump jump jump jump jump 손 높이 들다 jump 무릎 굽히다 jump 미소 활짝 박수 짝짝 모두 다 같이 jump 노래 부르다 jump 손 마주 잡고 jump 음악 맞추다 박수 짝짝 모두 다 같이 jump 신나다 jump 더 높이 jump 머리 가 저 구름 닿다 때 까지 뛰다 하나 둘 셋 웃다 하하 같이 라랄 라랄 라 라랄 라랄 라 준비 하다 ready jump jump jump jump jump jump jump jump jump jump jump ready jump jump jump jump jump jump jump jump jump jump jump ready jump jump jump jump jump jump jump jump jump jump jump ready jump jump jump jump jump jump jump jump jump jump jump 친구 모두 모이다 여기 여기 붙다 jump 신나다 발 굳다 jump jump jump jump jump 엄마 아빠 이모 고모 삼촌 뛰다 jump 누구 든 상관없다 jump jump jump jump jump 엉덩이 흔들다 jump 하늘 높이 Lets go jump 머리 가 찰랑 박수 짝짝 모두 다 같이 jump 하다 바퀴 빙 돌다 jump 눈 마주 치고 jump 뛰다 하다 발짝 박수 짝짝 모두 다 같이 jump 신나다 jump 더 높이 jump 머리 가 저 구름 닿다 때 까지 뛰다 하나 둘 셋 웃다 하하 같이 라랄 라랄 라 라랄 라랄 라 준비 하다 ready jump jump jump jump jump jump jump jump jump jump jump ready jump jump jump jump jump jump jump jump jump jump jump ready jump jump jump jump j

In [35]:
df.lyric = df.lyric.str.replace('[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣\n]', ' ', regex=True).str.strip()

In [36]:
lyrics = []
for lyric in df.lyric:
    lyric = lyric.replace('\n', ' ')
    morphs = okt.morphs(lyric, stem=True)
    tmp = [word for word in morphs if word not in stop_words]
    lyrics.append(' '.join(tmp))
df['morphs'] = lyrics

In [37]:
# 학습할 데이터열 생성
df['total'] = df.morphs + (' ' + df.title) + (' ' + df.artist) * 2 + (' ' + df.composer) * 2 + (' ' + df.lyricist) * 2 + (' ' + df.genre) * 3

In [38]:
df.set_index('songId', inplace=True)
df.reset_index(inplace=True)

In [39]:

from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)

In [10]:
# 장르가 힙합이나 댄스 : 결과 안 좋거나 거의 비슷하다. 
# from sklearn.feature_extraction.text import CountVectorizer
# cvect = CountVectorizer(stop_words='english')
# total_cv = cvect.fit_transform(df.total)

In [40]:
indices = pd.Series(df.index, index=df.songId)

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)

In [51]:
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_sim = cosine_similarity(total_cv)

In [42]:
def get_recommendation(songId, cos_sim):
    index = indices[songId]
    sim_scores = pd.Series(cos_sim[index])
    song_indices = sim_scores.sort_values(ascending=False).head(31).tail(30).index
    return df.songId.iloc[song_indices]

In [43]:
df[df.title.str.contains('성시경')]

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq,comment_like_total,morphs,total
3129,3853977,뜨거운 안녕 (Feat. 성시경),싸이 (PSY),랩/힙합,싸이6甲 Part.1,유희열,싸이 (PSY),20120715,https://cdnimg.melon.co.kr/cm/album/images/021...,107,79115,PSY collaboration\nTOY introducing 성발라\n불타 올랐던...,445029956 429291289,79222,PSY collaboration TOY introducing 성 발라 불타 오르다 ...,PSY collaboration TOY introducing 성 발라 불타 오르다 ...
4245,34997080,감동이야 (feat. 성시경),싸이 (PSY),랩/힙합,싸다9,싸이 (PSY),화이트99,20220429,https://cdnimg.melon.co.kr/cm2/album/images/10...,122,19021,넌 역시 감동이야 꿈을 꾸는 듯해\n넌 역시 감동이야 품에 품은 듯해 \n영원을 믿...,439324772 418156677,19143,넌 역시 감동 이야 꿈 꾸다 듯 하다 넌 역시 감동 이야 품 품다 듯 하다 영원 믿...,넌 역시 감동 이야 꿈 꾸다 듯 하다 넌 역시 감동 이야 품 품다 듯 하다 영원 믿...


In [44]:

# 키워드 바꿔가면서 찾아보기
key_title = '안녕'
key_artist = '성시경'
print(key_title, key_artist)

key_title = re.sub('['+string.punctuation+']', '', key_title)
key_artist = re.sub('['+string.punctuation+']', '', key_artist)

songIds =  df[df.title.str.replace('['+string.punctuation+']','', regex=True).str.contains(key_title, case=False) 
                & df.artist.str.replace('['+string.punctuation+']','', regex=True).str.contains(key_artist, case=False)][['songId', 'title', 'artist', 'album', 'date', 'img']]

songIds

안녕 성시경


,songId,title,artist,album,date,img
586,9646401,"다정하게, 안녕히",성시경,구르미 그린 달빛 OST Part.5,20160914,https://cdnimg.melon.co.kr/cm/album/images/027...
3729,1884329,안녕 나의 사랑,성시경,"여기, 내 맘속에...",20080612,https://cdnimg.melon.co.kr/cm/album/images/003...


In [45]:
find_songid = songIds.songId.values[0]
a = get_recommendation(find_songid, cosine_sim).to_frame()
a.head()

,songId
348,4446485
414,3079872
2605,8302
385,4864206
412,4300699


In [46]:

# df[df['songId'].isin(a.songId[1:6])].to_dict('records')
df[df['songId'].isin(a.songId[1:6])][['songId', 'title', 'artist', 'genre', 'album', 'lyricist', 'composer']]

,songId,title,artist,genre,album,lyricist,composer
144,1124448,선인장,성시경,발라드 국내영화,연애소설 OST,윤사라,조규만
385,4864206,"묘해, 너와",어쿠스틱 콜라보,발라드 국내드라마,연애의 발견 OST Part 4,심현보,심현보
412,4300699,너에게,성시경,발라드 국내드라마,응답하라 1994 OST Part 2,서태지,서태지
414,3079872,너는 나의 봄이다,성시경,발라드 국내드라마,시크릿 가든 OST Part.4,이미나,성시경
2605,8302,내게 오는 길,성시경,발라드,처음처럼,양재선,김형석


In [47]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
sim_scores = pd.Series(cosine_sim_total[indices[find_songid]])
sim_scores.sort_values(ascending=False).head(11).tail(10)

348     0.336221
414     0.242726
2605    0.201507
385     0.176536
412     0.174383
144     0.174015
2655    0.172468
2642    0.163746
3594    0.161197
3651    0.160714
dtype: float64

In [48]:
# 찾고 싶은 구간 정하기
numbers = df['comment_like_total']
sorted_numbers = np.sort(numbers)
q1 = np.percentile(sorted_numbers, 25)
q2 = np.percentile(sorted_numbers, 50)
filtered_data = df[(df['comment_like_total'] >= q1) & (df['comment_like_total'] < q2)]
filtered_data = filtered_data[['songId', 'comment_like_total']]
filtered_data.songId.values

array(['35837631', '35866717', '35879546', ..., '3080890', '2981597',
       '34183979'], dtype=object)

In [49]:
# 유사도 top 30 중 원하는 구간에 있는 songId 추출(유사도순)
d = a[a['songId'].isin(filtered_data.songId.values)].head(5)
d

,songId
144,1124448
590,35829148
373,36443231
55,418200


In [50]:
# filtered_df = df[df['songId'].isin(a.songId.values[:5])].to_dict('records')
filtered_df = df[df['songId'].isin(d.songId.values[:5])][['songId', 'title', 'artist', 'genre', 'album', 'lyricist', 'composer']]
filtered_df

,songId,title,artist,genre,album,lyricist,composer
55,418200,희재 (Edit Version),성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR
144,1124448,선인장,성시경,발라드 국내영화,연애소설 OST,윤사라,조규만
373,36443231,이 밤이 지나면,박민혜 (빅마마),발라드 국내드라마,닥터 차정숙 OST Part.4,김수빈(AIMING),김창락(AIMING)
590,35829148,오랫동안,성시경,발라드 국내드라마,오랫동안 (커튼콜 OST Part.5),Score (13),Score (13)
